In [ ]:
from recbole.quick_start import run_recbole

# 1. Declare model

In [ ]:
model = "BPR"

# 2. Declare data

In [ ]:
data = "ml-100k"

# 3. Start training

In [ ]:
run_recbole(model=model, dataset=data)

# 4. Start tuning hyper-param

# 5. Offline evaluation

In [ ]:
EVAL_MODES = {
    "temporal": "TO",
    "leave1out": "LS"
}

In [ ]:
selected_eval_models = [EVAL_MODES[k] for k in ["temporal", "leave1out"]]
selected_eval_metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
selected_topk = 10

eval_args = {
    "order": "TO",
    "split": {
        'LS': "valid_and_test"
    },
    "group_by": None
}

config_dict = {
    'eval_args': eval_args,
    # 'train_neg_sample_args': None,
}

run_recbole(model=model, dataset=data, config_dict=config_dict)

In [ ]:
run_recbole(model='NPE', dataset='ml-100k', config_file_list=['test.yml'])